<a href="https://colab.research.google.com/github/markwang0/GeoFlood/blob/WIP-pygeoflood-class/examples/geoflood_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install in-development branch of GeoFlood

In [3]:
%%capture
!pip install git+https://github.com/markwang0/geoflood@WIP-pygeoflood-class

### Download `OC1mTest.tif` and upload it to this runtime's file system in the `/content` directory
https://utexas.box.com/s/d6j0oy0cw2fwg0mp8w60aw9rljw880t0

### Import Pygeoflood class, instantiate model with path to DEM, run GeoFlood operations

In [5]:
# instantiate pygeoflood object with path to DEM
from pygeoflood import Pygeoflood

# only nonlinear filter implemented so far
my_dem_path = "/content/OC1mTest.tif"
my_geoflood_model = Pygeoflood(dem_path=my_dem_path)
my_geoflood_model.run_nonlinear_filter()

# the Perona-Malik filtered DEM is saved as [DEM_name]_PM_filtered.tif



Computing slope of raw DTM
DEM slope array shape: (3378, 4750)
Computing lambda = q-q-based nonlinear filtering threshold
('dem smoothing Quantile', 0.9)
('edgeThresholdValue:', 0.34356024861335754)
time taken to complete nonlinear filtering: 22.0 seconds
